# assignable types


The only real limitation of user defined types compared to CoHDL builtins is, that they cannot be type qualified. So while it is possible to implement a class for fixed point arithmetic, you are not allowed to create a Signal of such a type. 

Instead, classes can provide overloads for the three assignment operators and mimic the behavior of Signals and Variables.

In [5]:
from cohdl import Port, BitVector
from cohdl import Entity

from cohdl import std

class Coord:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    # all assignment operators must return
    # the self object because otherwise the
    # result name would be overridden
    # which is not allowed in CoHDL

    def __ilshift__(self, other):
        self.x <<= other.x
        self.y <<= other.y
        return self
    
    def __ixor__(self, other):
        self.x ^= other.x
        self.y ^= other.y
        return self
    
    def __imatmul__(self, other):
        self.x @= other.x
        self.y @= other.y
        return self

class MyEntity(Entity):
    x_in = Port.input(BitVector[32])
    y_in = Port.input(BitVector[32])

    x_out = Port.output(BitVector[32])
    y_out = Port.output(BitVector[32])

    def architecture(self):
        coord_in = Coord(self.x_in, self.y_in)
        coord_out = Coord(self.x_out, self.y_out)

        @std.sequential
        def logic():
            nonlocal coord_out

            # coord is not a signal
            # but can be assigned like one
            coord_out <<= coord_in

vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)


library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    x_in : in std_logic_vector(31 downto 0);
    y_in : in std_logic_vector(31 downto 0);
    x_out : out std_logic_vector(31 downto 0);
    y_out : out std_logic_vector(31 downto 0)
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_x_out : std_logic_vector(31 downto 0);
  signal buffer_y_out : std_logic_vector(31 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  x_out <= buffer_x_out;
  y_out <= buffer_y_out;
  

  logic: process(x_in, y_in)
  begin
    buffer_x_out <= x_in;
    buffer_y_out <= y_in;
  end process;
end architecture arch_MyEntity;


To avoid the code duplication seen in the last example, Signals and Variables have an `_assign_` method as an alternative to the assignment operators and properties. It takes two arguments, the value to assign and an instance of the enumeration `cohdl.AssignMode`.

In [6]:
from cohdl import Signal, Variable, Bit
from cohdl import AssignMode

sig = Signal[Bit](False)
var = Variable[Bit]()

# each of the three assignment operators
# can be written in three equivalent ways

sig <<= True
sig.next = True
sig._assign_(True, AssignMode.NEXT)

sig ^= True
sig.push = True
sig._assign_(True, AssignMode.PUSH)

var @= True
var.value = True
var._assign_(True, AssignMode.VALUE)

# compilation error because
# AssignMode must match type qualifier
# sig._assign_(True, AssignMode.VALUE)
# var._assign_(True, AssignMode.NEXT)
# var._assign_(True, AssignMode.PUSH)

Using the `_assign_` method we can describe the required member assignments in a single method and reuse it. The class `Assignable` automates this by forwarding all assignments to an abstract method.

The CoHDL standard library already contains an implementation of this class `std.Assignable` that also defines the .next/.push/.value properties).

In [7]:
from cohdl import Port, BitVector
from cohdl import Entity

from cohdl import std
from cohdl import AssignMode
import cohdl

class Assignable:

    # there is nothing special about the name _assign_
    # it is only used for consistency with the
    # _assign_ method of type qualified objects
    def _assign_(self, other, mode: AssignMode):
        raise AssertionError("_assign_ must be overridden by child class")

    def __ilshift__(self, other):
        self._assign_(other, AssignMode.NEXT)
        return self
    
    def __ixor__(self, other):
        self._assign_(other, AssignMode.PUSH)
        return self
    
    def __imatmul__(self, other):
        self._assign_(other, AssignMode.VALUE)
        return self

class Complex(Assignable):
    def __init__(self, r, i):
        self.r = r
        self.i = i
    
    def _assign_(self, other, mode: AssignMode):
        # isinstance is a cohdl intrinsic
        if isinstance(other, Complex):
            self.r._assign_(other.r)
            self.i._assign_(other.i)
        else:
            self.r._assign_(other)
            self.i._assign_(0)

class MyEntity(Entity):
    x_in = Port.input(BitVector[32])
    y_in = Port.input(BitVector[32])

    x_out = Port.output(BitVector[32])
    y_out = Port.output(BitVector[32], default=cohdl.Null)

    def architecture(self):
        coord_in = Coord(self.x_in, self.y_in)
        coord_out = Coord(self.x_out, self.y_out)
        coord_variable = Coord(Variable[BitVector[32]](), Variable[BitVector[32]]())

        @std.sequential
        def logic():
            nonlocal coord_out, coord_variable
            
            # assign input to variable
            # and variable to output
            coord_variable @= coord_in
            coord_out <<= coord_variable


vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    x_in : in std_logic_vector(31 downto 0);
    y_in : in std_logic_vector(31 downto 0);
    x_out : out std_logic_vector(31 downto 0);
    y_out : out std_logic_vector(31 downto 0)
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_x_out : std_logic_vector(31 downto 0);
  signal buffer_y_out : std_logic_vector(31 downto 0) := "00000000000000000000000000000000";
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  x_out <= buffer_x_out;
  y_out <= buffer_y_out;
  

  logic: process(x_in, y_in)
    variable var : std_logic_vector(31 downto 0);
    variable var_1 : std_logic_vector(31 downto 0);
  begin
    var := x_in;
    var_1 := y_in;
    buffer_x_out <= var;
    buff

As the following `Complex` class demonstrates assignments of different types are also possible. For CoHDL `_assign_` is a normal method with no further restrictions.

In [8]:
class Complex(Assignable):
    def __init__(self, r, i):
        self.r = r
        self.i = i
    
    def _assign_(self, other, mode: AssignMode):
        # isinstance is a cohdl intrinsic
        if isinstance(other, Complex):
            self.r._assign_(other.r)
            self.i._assign_(other.i)
        else:
            self.r._assign_(other)
            self.i._assign_(0)